<div>
<img src="https://www.nebrija.com/images/logos/logotipo-universidad-nebrija.jpg" width="200">
</div>

**ALGORITMOS** -
Prof: Carmen Pellicer Lostao


# Hardware Cuantico real

---








Cargamos las credenciales de nuestra cuenta e importamos las librerias iniciales

In [ ]:
from qiskit import IBMQ
#si falla el load_account() ejecutar antes un save_account()
#IBMQ.save_account('YOUR API KEY', overwrite=True)
provider = IBMQ.load_account()

Un [provider](https://docs.quantum.ibm.com/api/qiskit/providers) es un objeto que proporciona servicios externos a Terra, por ejemplo proporciona backends para la ejecucion de los sercivios.

Cuando cargamos nuestra cuenta podemos acceder a una serie de backends a través de la clase `Provider` de qiskit

### EJERCICIO

Itera `provider.backends()` e imprime por pantalla el resultado de sus metodos `.name()`, `.configuracion().n_qubits`

Podemos usar la instruccion dir(obj) y vars(obj) para saber las propiedades y metodos que tiene el ultimo de los providers:

- Utiliza dir() para obtener una lista de todos los atributos (incluidos los métodos) del objeto.
- Utiliza vars() para obtener la propiedad __dict__ del objeto, que contiene variables de instancia

Prueba dir() sobre el objeto y sobre obj.configuracion()

Encuentra el conjunto de puertas base de cada backend

## Ejecutando circuitos en HW cuántico: compilacion de circuitos

Los backends a los que tenemos acceso son:

In [ ]:
[(b.name(), b.configuration().n_qubits) for b in provider.backends()]

In [ ]:
# si se tiene acceso a mas de un hub:
#provider = IBMQ.get_provider("ibm-q-internal")
#[(b.name(), b.configuration().n_qubits) for b in provider.backends()]

Los vemos todos y consultamos cual es que tiene menos trabajos en cola (least busy)

Podemos usar la funcion `least_busy` para seleccionar el backend menos cargado en cada momento:

In [ ]:
from qiskit.providers.ibmq import least_busy

backend = least_busy(provider.backends(
                simulator=False,
                filters=lambda b: b.configuration().n_qubits >= 2))
backend

### EJECUCION DE UN CIRCUITO

Creamos nuestro circuito de 2 qubits, un estado de Bell

In [ ]:
from qiskit import QuantumCircuit
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.measure_all()
circuit.draw()

Lo ejecutamos siempre primero en un simulador

In [ ]:
from qiskit import Aer
from qiskit.visualization import plot_histogram
sim = Aer.get_backend('aer_simulator')
result = sim.run(circuit).result()
counts = result.get_counts()
plot_histogram(counts)

In [ ]:
#el metodo run sobre el backend crea un job 
job = sim.run(circuit)

In [ ]:
job.result()

In [ ]:
circuit.draw('mpl')

Vamos a lanzar la ejecucion de nuestro circuito al backend que tiene menos jobs en cola.

In [ ]:
backend = least_busy(provider.backends(
                simulator=False,
                filters=lambda b: b.configuration().n_qubits >= 2))
backend

Necesitamos transpilar el circuito al backend real

In [ ]:
from qiskit import transpile

transpiled_circuit = transpile(circuit, backend)
transpiled_circuit.draw(idle_wires=False, fold=-1) #sin idle wires y todo en una linea

Notese que `transpiling` puede no tomar un backend y entonces simplemente no hace ninguna operacion de compilado del circuito.

In [ ]:
notranspiled_circuit = transpile(circuit)
notranspiled_circuit.draw(idle_wires=False, fold=-1) #la opcion fold controla la paginacion al dibujar el circuito, es este caso con valor -1 esta desactivada y idle_wires quita qubits no usados

Para el seguimiento de la ejecucion utilizamos la utilidad `job_watcher`, que nos va diciendo como estamos en la cola de forma interactiva

In [ ]:
from qiskit.tools.jupyter import *
%qiskit_job_watcher  #lanzamos el widget de job_watcher para hacer seguimiento

Ejecutamos el circuito

In [ ]:
job = backend.run(transpiled_circuit)

Para ver el estado de ejecucion:

In [ ]:
job.status()

In [ ]:
result = job.result()
counts = result.get_counts()
plot_histogram(counts)

Cuando ha terminado obtenemos los resultados y podemos ver como se ha ejecutado nuestro circuito en el backend visualizando los qubits utilizados:

In [ ]:
from qiskit.visualization import plot_circuit_layout, plot_gate_map
display(plot_gate_map(backend))
plot_circuit_layout(transpiled_circuit, backend)

Podemos ver como se ha ejecutado el circuito en los qubits del dispositivo y el mapa de puertas que se ha utilizado:

In [ ]:
from qiskit.visualization import plot_circuit_layout, plot_gate_map

display(transpiled_circuit.draw(idle_wires=False))
display(plot_gate_map(backend))
plot_circuit_layout(transpiled_circuit, backend)

Tambien podemos ver los resutados a partir de un Job ID que ya se haya ejecutado

In [ ]:
backend = provider.get_backend('ibm_kyoto')
job = backend.retrieve_job('cpjnktz6q5h0008bjcn0') # otro en ibm_kyoto-> cpkv0mt6q5h0008bmxtg

result = job.result()
counts = result.get_counts()
plot_histogram(counts)

#### EJERCICIO

Creamos un circuito de 3  qubits y lo ejecutamos siguiendo los pasos siguientes:

- calculamos el backend menos ocupado
- transpilamos el circuito a ese backend, con o sin indicarle un `initial_layout`
- dibujamos el layout de como se ha desplegado nuestro circuito en el backend
- dibujamos el mapa de puertas del backend
- lo ejecutamos y obtenemos los resultados

También podemos especificar el `layout` que se utiliza para desplegar nuestro circuito en el dispositivo. 

In [ ]:
transpiled_circuit = transpile(circuit, backend, initial_layout=[1, 2, 3]) #elige como qubits 0,1,2 de nuestro circuito -> los qubits del dispositivo 1,2,3 por ejemplo
display(transpiled_circuit.draw(idle_wires=False))  #para evitar muchos qubits no usados en la representacion
display(plot_circuit_layout(transpiled_circuit, backend))
plot_gate_map(backend)

#### EJERCICIO

Comprueba la complejidad del circuito transpilado anterior, si no elegimos qubits que estén conectados

#### EJERCICIO

Para un circuito dado, elije el backend de ejecucion menos ocupado y transpilalo para su ejecucion.

Mapea tres qubits que esten conectados y dibuja el circuito resultante que se ejecutará en el ordenador cuántico.

In [ ]:
# Creamos este circuito
circuit = QuantumCircuit(3)
circuit.h([0,1,2])
circuit.ccx(0, 1, 2)
circuit.h([0,1,2])
circuit.ccx(2, 0, 1)
circuit.h([0,1,2])
circuit.measure_all()
circuit.draw()

In [ ]:
#Buscamos el Backend menos cargado


In [ ]:
#Transpilamos nuestro circuito para este Backend


Observamos la complejidad del circuito, si no elegimos qubits que estén conectados

In [ ]:
#Dibujamos como se ha desplegado el circuito en el backend y el mapa de puertas que hay disponibles para conectar los qubits


## Otras opciones de Backend

### Nivel de optimizacion

Podemos dejar que qiskit optimice el layout de nuestro circuito en el hardware cuantico. Los niveles más altos son niveles de mayor nivel de optimizacion a costa de un tiempo mayor para el transpiling

 * 0: sin optimizacion, mas alla de un despliegue con un mapeo especifico en el backend
 * 1: optimizacion ligera, colapsando puertas adyacentes en los qubits cercanos (default)
 * 2: optimizacion media con mapeado adaptativo de qubits, teniendo en cuenta el ruido de los qubits y las puertas y posibles efectos que lo aminoren o compensen entre si.
 * 3: optimizacion alta con mapeo adatativo de ruido de los qubits y puertas, para buscar que se aminore y compense
 
Veamos un ejemplo de utilizacion de este nivel de optimizacion:

In [ ]:
circuit = QuantumCircuit(3)
circuit.h(0)
circuit.cx(0, 1)
circuit.cx(1,2)
circuit.cx(0,2)
circuit.measure_all()
circuit.draw()

In [ ]:
from qiskit import transpile

transpiled_circuit = transpile(circuit, backend,optimization_level=0)
transpiled_circuit.draw( idle_wires=False)
print(transpiled_circuit.count_ops(),transpiled_circuit.depth())

#### EJERCICIO

Comprueba la complejidad del circuito transpilado en los cuatro niveles de optimizacion disponibles

La operacion de transpiling es un proceso estocástico, esto es, algo aleatorio y para que sea repetible podemos utilizar la opcion de fijar la semilla de inicio.

Prueba diferentes valores `seed_transpiler=0` y `seed_transpiler=11`  por ejemplo

### Las puertas base que soporta el dispositivo


Veamos las puestas de base del Backend

In [ ]:
backend.configuration().basis_gates

Podemos forzar a hacer un transpiling con unas determinadas puertas de base

In [ ]:
transpiled = transpile(circuit, basis_gates=['x', 'cx', 'h', 'p'])
transpiled.draw(fold=-1)

#### EJERCICIO

Dado un circuito, prueba hacer un transpilado al backend cuantico que no contenga alguna de las puertas base del backend

In [ ]:
circuit = QuantumCircuit(3)
circuit.h(0)
circuit.cx(0, 1)
circuit.cx(1,2)
circuit.cx(0,2)
circuit.measure_all()
circuit.draw()

### Controlando el despliegue del circuito en el dispositivo con `CouplingMap` e `initial_layout`

Podemos comprobar el `coupling_map` del backend de ejecucion cuantico

In [ ]:
backend.configuration().coupling_map

#### EJERCICIO

Investiga las opciones de la funcion [`transpile`](https://docs.quantum.ibm.com/api/qiskit/0.28/qiskit.compiler.transpile#qiskitcompilertranspile)

Dado un circuito, prueba diferentes opciones de transpilacion:

- coupling_map -> [(0,1),(1,2)] 
- initial_layout -> [1, 0, 2], 
- basis_gates -> ['x', 'cx', 'h', 'p'], 
- optimization_level -> 3, 
- approximation_degree -> 0.99

y observa la complejidad del circuito transpilado midiendo las operaciones y la profundidad resultates.
Dibuja tambien el circuito.


In [ ]:
circuit = QuantumCircuit(3)
circuit.h(0)
circuit.cx(0, 1)
circuit.cx(1,2)
circuit.cx(0,2)
circuit.measure_all()
circuit.draw()

In [ ]:
from qiskit.transpiler import CouplingMap

cm=CouplingMap([(0,1),(1,2)])
               

transpiled =
            
print(transpiled.count_ops(), transpiled.depth())
display(transpiled.draw(fold=-1,idle_wires=False))

## IonQ con Qiskit

Podemos ejecutar otro sofgware, como por ejemplo el de IonQ desde Qiskit

In [ ]:
!pip install qiskit-ionq

In [ ]:
from qiskit_ionq import IonQProvider
provider = IonQProvider(<your token>)

In [ ]:
[(b.name(), b.configuration().n_qubits) for b in provider.backends()]

In [ ]:
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.measure_all()
circuit.draw()

In [ ]:
backend = provider.get_backend("ionq_qpu")
job = backend.run(circuit)

In [ ]:
plot_histogram()

In [ ]:
job.get_counts()

In [ ]:
plot_histogram(job.get_counts())